In [76]:
import pandas as pd
from confluent_kafka import SerializingProducer
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroSerializer
from confluent_kafka.serialization import StringSerializer

#Load csv into pandas dataframe
df = pd.read_csv('olist_orders_dataset.csv')

# Examine it's structure and contents
print(df.head())

# print(df.info())

# print(df.describe())

# print(df.columns)

# print(df.shape)

def delivery_report(err, msg):
    """
    Reports the failure or success of a message delivery.

    Args:
        err (KafkaError): The error that occurred on None on success.

        msg (Message): The message that was produced or failed.

    Note:
        In the delivery report callback the Message.key() and Message.value()
        will be the binary format as encoded by any configured Serializers and
        not the same object that was passed to produce().
        If you wish to pass the original object(s) for key and value to delivery
        report callback we recommend a bound callback or lambda where you pass
        the objects along.

    """
    if err is not None:
        print("Delivery failed for record {}: {}".format(msg.key(), err))
        return
    print('Record {} successfully produced to {} [{}] at offset {}'.format(
        msg.key(), msg.topic(), msg.partition(), msg.offset()))
    
# Define Kafka configuration
kafka_config = {
    'bootstrap.servers': 'pkc-6ojv2.us-west4.gcp.confluent.cloud:9092',
    'sasl.mechanisms': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': 'KE3YCSOIN46EIKLM',
    'sasl.password': 'DfHmXuw2vR38KYDSbvXyGBNZCxsZrFF+VtTTPoeZMgp33o/oV8l7s0ObJ/Q48HmK'
}

# Create a Schema Registry client
schema_registry_client = SchemaRegistryClient({
  'url': 'https://psrc-zj6ny.us-east-2.aws.confluent.cloud',
  'basic.auth.user.info': '{}:{}'.format('NJHM72JKLO43DSHW', '3jAGYLSdojChjA9uv3fuaV/dS7D29l7tcZYcEVmDQqD0IeXeyCpNOXa7lTZb4hpy')
})

# Fetch the latest Avro schema for the value
subject_name = 'olist_orders_dataset-value'
schema_str = schema_registry_client.get_latest_version(subject_name).schema.schema_str
# print(schema_str)

# Create Avro Serializer for the value
# key_serializer = AvroSerializer(schema_registry_client=schema_registry_client, schema_str='{"type": "string"}')
key_serializer = StringSerializer('utf_8')
avro_serializer = AvroSerializer(schema_registry_client, schema_str)


# Define the SerializingProducer
producer = SerializingProducer({
    'bootstrap.servers': kafka_config['bootstrap.servers'],
    'security.protocol': kafka_config['security.protocol'],
    'sasl.mechanisms': kafka_config['sasl.mechanisms'],
    'sasl.username': kafka_config['sasl.username'],
    'sasl.password': kafka_config['sasl.password'],
    'key.serializer': key_serializer,  # Key will be serialized as a string
    'value.serializer': avro_serializer  # Value will be serialized as Avro
})

count = 0
# Iterate over DataFrame rows and produce to Kafka
for index, row in df.iterrows():
    if count < 50:
        key = f"{row['customer_id']}_{row['order_id']}"
        # Replace NaN values with None in the DataFrame
        row = row.where(pd.notna(row), None)

        # Create a dictionary from the row values
        value = row.to_dict()
        # print(value)
        # Produce to Kafka
        producer.produce(topic='olist_orders_dataset', key=str(key), value=value, on_delivery=delivery_report)
        producer.flush()
        count += 1

print("Data successfully published to Kafka")

                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
2    delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   
3    delivered      2017-11-18 19:28:06  2017-11-18 19:45:59   
4    delivered      2018-02-13 21:18:39  2018-02-13 22:20:29   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 21:25:13   
1          2018-07-26 14:31:00           2018-08

Record b'7f2178c5d771e17f507d3c1637339298_2ce1ad82022c1ba30c2079502ac725aa' successfully produced to olist_orders_dataset [0] at offset 24
Data successfully published to Kafka
